# Урок 6. Расчёт длительности А/B теста + продвинутые топики

### Задача 1.
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156. Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик 20000 человек в день. Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% 

In [2]:
import numpy as np
from statsmodels.stats.power import tt_ind_solve_power

def calculate_experiment_duration(mean, sd, effect_size, alpha, beta, daily_traffic):
    """
    Функция для расчета длительности A/B теста.

    :param mean: Среднее значение метрики в контрольной группе.
    :param sd: Стандартное отклонение метрики.
    :param effect_size: Ожидаемое изменение метрики в экспериментальной группе.
    :param alpha: Уровень значимости (вероятность ошибки первого рода).
    :param beta: Вероятность ошибки второго рода (1 - мощность теста).
    :param daily_traffic: Среднедневная посещаемость.
    
    :return: Количество дней для проведения эксперимента.
    """
    # Расчет ожидаемого изменения
    effect_size_absolute = effect_size * mean
    
    # Расчет размера эффекта
    effect_size_standardized = effect_size_absolute / sd
    
    # Расчет размера выборки на группу
    sample_size_per_group = tt_ind_solve_power(effect_size=effect_size_standardized, 
                                               alpha=alpha, 
                                               power=1-beta, 
                                               ratio=1, 
                                               alternative='two-sided')
    
    # Общий размер выборки
    total_sample_size = sample_size_per_group * 2
    
    # Расчет длительности эксперимента
    duration_in_days = np.ceil(total_sample_size / daily_traffic)
    
    return duration_in_days

# Параметры эксперимента
mean_spent_time = 25  # Средний таймспент на пользователя в день
sd_spent_time = 156   # Стандартная ошибка
expected_change = 0.1  # Ожидаемое изменение таймспента в новой версии
alpha_level = 0.05    # Уровень значимости
beta_level = 0.2      # Ошибка второго рода
daily_users = 20000   # Средний дневной трафик

# Расчет длительности эксперимента
experiment_duration = calculate_experiment_duration(mean_spent_time, sd_spent_time, 
                                                    expected_change, alpha_level, 
                                                    beta_level, daily_users)

# Вывод результата
print(f"Необходимая длительность эксперимента: {experiment_duration} дней")

Необходимая длительность эксперимента: 7.0 дней


### Задача 2.
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта. Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.Средний трафик 4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

In [6]:
import numpy as np
from statsmodels.stats.power import GofChisquarePower

def calculate_sample_size_and_duration(p1, p2, alpha, beta, daily_traffic):
    """
    Функция для расчета размера выборки и длительности A/B теста на основе изменения bounce rate.

    :param p1: Текущий bounce rate.
    :param p2: Ожидаемый bounce rate после изменений.
    :param alpha: Уровень значимости (вероятность ошибки первого рода).
    :param beta: Вероятность ошибки второго рода (1 - мощность теста).
    :param daily_traffic: Средний дневной трафик.
    
    :return: Количество дней для проведения эксперимента.
    """
    # Расчет эффекта
    effect_size = abs(p2 - p1)
    
    # Определение объекта для расчета мощности
    power_analysis = GofChisquarePower()
    
    # Расчет размера выборки на группу
    sample_size = power_analysis.solve_power(effect_size=effect_size, nobs=None,
                                             alpha=alpha, power=1-beta)
    
    # Общий размер выборки
    total_sample_size = sample_size * 2
    
    # Расчет длительности эксперимента
    duration_in_days = np.ceil(total_sample_size / daily_traffic)
    
    return int(duration_in_days)  # Преобразование в целое число

# Параметры эксперимента
current_bounce_rate = 0.40  # Текущий bounce rate
expected_improvement = 0.20 # Ожидаемое улучшение
expected_bounce_rate = current_bounce_rate - current_bounce_rate * expected_improvement  # Ожидаемый bounce rate
alpha_level = 0.05  # Уровень значимости
beta_level = 0.20   # Ошибка второго рода
daily_users = 4000  # Средний дневной трафик

# Расчет длительности эксперимента
experiment_duration = calculate_sample_size_and_duration(current_bounce_rate, expected_bounce_rate, 
                                                         alpha_level, beta_level, daily_users)

# Вывод результата
print(f"Необходимая длительность эксперимента: {experiment_duration} дней")

Необходимая длительность эксперимента: 1 дней
